# Entrenamiento del Modelo con PySpark

Este notebook implementa el entrenamiento del modelo usando PySpark y TensorFlow.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import VectorAssembler
import json
from datetime import datetime

# Iniciar Spark con más memoria para procesamiento de imágenes
spark = SparkSession.builder \
    .appName("ModelTraining") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "10") \
    .getOrCreate()

# Configurar paths
PROCESSED_DIR = 'data/processed'
LABELS_FILE = 'data/labels.json'
MODEL_DIR = 'data/models'
os.makedirs(MODEL_DIR, exist_ok=True)

In [ ]:
# Cargar datos procesados
print("Cargando datos procesados...")
df = spark.read.parquet(os.path.join(PROCESSED_DIR, "processed_pages.parquet"))
print(f"Total de registros cargados: {df.count()}")

# Cargar etiquetas
print("\nCargando archivo de etiquetas...")
with open(LABELS_FILE, 'r') as f:
    labels = json.load(f)
print(f"Etiquetas cargadas para {len(labels)} documentos")

# Función para determinar si una página es primera página
def is_first_page(pdf_name, page_number):
    if pdf_name in labels:
        return 1 if page_number in labels[pdf_name]["target_pages"] else 0
    return 0

# Registrar UDF
is_first_page_udf = spark.udf.register("is_first_page", is_first_page)

# Añadir etiquetas al DataFrame
print("\nAsignando etiquetas a los datos...")
df = df.withColumn("label", 
    is_first_page_udf(col("pdf_name"), col("page_number")))

# Mostrar distribución de etiquetas
print("\nDistribución de etiquetas:")
df.groupBy("label").count().show()

In [ ]:
# Preparar datos para entrenamiento
print("Preparando datos para entrenamiento...")

# Convertir a numpy arrays para TensorFlow
train_data = df.select("features", "label").toPandas()
X = np.array([np.array(x) for x in train_data["features"]])
y = train_data["label"].values

# Dividir en entrenamiento y validación
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Mantener proporción de clases
)

print(f"\nDimensiones de los datos:")
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_train: {np.bincount(y_train)}")
print(f"y_val: {np.bincount(y_val)}")

In [ ]:
def create_model(input_shape=(224*224,)):
    """Crea y retorna el modelo CNN"""
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Reshape((224, 224, 1)),
        
        # Primer bloque convolucional
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        # Segundo bloque convolucional
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        # Tercer bloque convolucional
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        # Capas densas
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    # Compilar modelo
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.AUC()
        ]
    )
    
    return model

# Crear y mostrar resumen del modelo
print("Creando modelo...")
model = create_model()
model.summary()

In [ ]:
# Configurar callbacks
checkpoint_path = os.path.join(MODEL_DIR, "checkpoints")
os.makedirs(checkpoint_path, exist_ok=True)

callbacks = [
    # Guardar mejor modelo
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(checkpoint_path, "model_{epoch:02d}-{val_loss:.2f}.h5"),
        save_best_only=True,
        monitor='val_loss'
    ),
    # Detener si no hay mejora
    tf.keras.callbacks.EarlyStopping(
        patience=5,
        restore_best_weights=True,
        monitor='val_loss'
    ),
    # Reducir learning rate si no hay mejora
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-6
    ),
    # TensorBoard logging
    tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(MODEL_DIR, 'logs', datetime.now().strftime("%Y%m%d-%H%M%S")),
        histogram_freq=1
    )
]

# Entrenar modelo
print("Iniciando entrenamiento...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=callbacks,
    class_weight={  # Manejar desbalance de clases
        0: 1.,
        1: np.sum(y_train == 0) / np.sum(y_train == 1)
    }
)

In [ ]:
# Guardar modelo final
print("Guardando modelo y resultados...")
model_path = os.path.join(MODEL_DIR, "final_model.h5")
model.save(model_path)
print(f"Modelo guardado en: {model_path}")

# Guardar historia de entrenamiento
history_path = os.path.join(MODEL_DIR, "training_history.pkl")
with open(history_path, "wb") as f:
    pickle.dump(history.history, f)
print(f"Historia de entrenamiento guardada en: {history_path}")

# Mostrar resultados finales
print("\nResultados finales del entrenamiento:")
for metric in history.history:
    if not metric.startswith('val_'):
        print(f"{metric}: {history.history[metric][-1]:.4f} "
              f"(validación: {history.history['val_'+metric][-1]:.4f})")

# Limpiar sesión de Spark
spark.stop()